In [2]:
import re
import requests
from bs4 import BeautifulSoup
import time
import numpy as np
import glob
import tqdm

## Data Scraping

In [5]:
def get_lyrics(main_url):
    # proxy = next(proxyPool)
    time.sleep(np.random.randint(10) + 10)
    req = requests.get(main_url, headers={"User-Agent":"Mozilla/5.0"})
    soup = BeautifulSoup(req.text, "html.parser")

    lyrics  = soup.get_text().split('Lyrics')[3].strip('\n').split('Submit')[0]
    if "(originally by" in lyrics: return ''
    lyrics = lyrics.replace('\r', '')
    lyrics = re.sub(r'(\n\n+)', '\n\n', lyrics)
    
    return lyrics[:-1]

def get_artist_links(main_url):
    # proxy = next(proxyPool)
    time.sleep(np.random.randint(10))
    req = requests.get(main_url, headers={"User-Agent":"Mozilla/5.0"})
    soup = BeautifulSoup(req.text, "html.parser")
    links = soup.find_all('a')
    data = soup.find_all('div',attrs={"id": "listAlbum"})
    all_links = []
    for div in data:
        links = div.find_all('a')
        for link in links:
            link_ref = link.get('href')
            if "https://www.azlyrics.com" in link_ref: continue
            all_links.append("https://www.azlyrics.com" + link_ref)
    return all_links

def get_all_artist_lyrics(main_url, outfile_name):
    links = get_artist_links(main_url)
    lyrics = ''
    with open(outfile_name, 'a') as outfile:
        for link in tqdm.tqdm(links):
            # print(link)
            try:
                lyrics = get_lyrics(link)
                outfile.write(lyrics)
            except Exception as e:
                print(e)

    return lyrics

In [9]:
artists = ["lorde"]
for artist in artists:
    print(artist)
    website = "https://www.azlyrics.com/" + artist[0] + '/' + artist + '.html'
    get_all_artist_lyrics(website, f'data/{artist}.txt')
    # with open(, 'w+') as f:
    #     f.write(lyrics)

lorde


100%|██████████| 62/62 [15:31<00:00, 15.02s/it]


In [9]:
filenames = glob.glob('data/*txt')
with open('data/all_lyrics.txt', 'w') as outfile:
    for fname in filenames:
        if 'all_lyrics' in fname: continue
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)

with open('data/all_lyrics.txt', 'r') as f:
    text = f.read()

    

with open('data/all_lyrics.txt', 'w+') as f:
    list_of_chars = "[^\n\"$&\'(),-.0123456789ABCDEFGHIJKLMNOPQRSTUVWY\[\]abcdefghijklmnopqrstuvwxyz ]"
    regex = re.compile(list_of_chars)
    x = regex.sub('', text)

    regex = re.compile("[<>:/\|?*]")
    x = regex.sub('', x)

    f.write(x)

with open('data/all_lyrics.txt', 'r', encoding='utf-8') as f:
    text = f.read()

len(text)

821319

In [10]:
n = int(len(text)*0.8)
train = text[:n]
val = text[n:]

In [ ]:
with open('data_splits/train.txt', 'w+') as f:
    f.write(train)
with open('data_splits/val.txt', 'w+') as f:
    f.write(val)

In [12]:
len(val)

164264

In [13]:
'>' in val

False

In [19]:
[x in train for x in '< > : / \ | ? *'.split(' ')]

[False, False, False, False, False, False, False, False]